In [66]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
morph = MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ольга\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
string.punctuation += '«»—–…“”•№→’€'
symbols = string.punctuation + '1234567890'

In [6]:
stops = set(stopwords.words('russian')) | {'gt', 'который', 'это'}

def remove_tags(text):
    text = re.sub(r'<[^>]+>', '', text)
    return text

def tokenize(text):
    words = [word.strip(symbols) for word in text.split()]
    words = [word for word in words if word and not ';' in word]
    return words

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

In [8]:
texts = open('habr_texts.txt', encoding = 'utf-8').read().splitlines()

In [9]:
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

# LDA

In [41]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [42]:
dct = gensim.corpora.Dictionary(ngrammed_texts)

In [43]:
dct.filter_extremes(no_above=0.1, no_below=20)
dct.compactify()

In [44]:
print(dct)

Dictionary(7139 unique tokens: ['address', 'architecture', 'assembly', 'await', 'azure']...)


In [70]:
corpus = [dct.doc2bow(text) for text in ngrammed_texts]

In [71]:
lda2 = gensim.models.LdaModel(corpus, 100, id2word=dct, passes=10)

In [99]:
print(lda2.show_topic(4, topn = 10))
print('\n')
print(lda2.show_topic(11, topn = 10))
print('\n')
print(lda2.show_topic(30, topn = 10))

[('космический', 0.055904854), ('аппарат', 0.051437534), ('земля', 0.042905286), ('спутник', 0.03724272), ('марс', 0.03693666), ('орбита', 0.036578394), ('полёт', 0.032689083), ('луна', 0.03160382), ('космос', 0.028491067), ('планета', 0.026907949)]


[('пациент', 0.03764556), ('кожа', 0.035166435), ('учёный', 0.0325364), ('человеческий', 0.020528192), ('врач', 0.020210983), ('болезнь', 0.019733092), ('лаборатория', 0.017683418), ('организм', 0.016903874), ('лечение', 0.015612155), ('заболевание', 0.015045924)]


[('товар', 0.096954726), ('магазин', 0.058286164), ('скидка', 0.048378527), ('покупка', 0.045971315), ('покупатель', 0.043772113), ('amazon', 0.040300988), ('доставка', 0.035668273), ('заказ', 0.031992152), ('потребитель', 0.024883162), ('продавец', 0.024341626)]


Метрики модели:

In [74]:
lda2.log_perplexity(corpus[:10000])

-7.78874924421697

In [83]:
coherence_model_lda2 = gensim.models.CoherenceModel(model=lda2, 
                                                  texts=texts, 
                                                   dictionary=dct, coherence='c_v')

topics = []
for topic_id, topic in lda2.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda2 = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dct, coherence='c_v')

In [84]:
coherence_model_lda2.get_coherence()

0.5474442339522236

# LDA with tfidf

In [64]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dct)
corpus = tfidf[corpus]

In [67]:
lda3 = gensim.models.LdaModel(corpus, 100, id2word=dct, passes=10)

In [101]:
print(lda3.show_topic(36, topn = 10))
print('\n')
print(lda3.show_topic(52, topn = 10))
print('\n')
print(lda3.show_topic(65, topn = 10))

[('мозг', 0.08137182), ('клетка', 0.0603882), ('боль', 0.056431685), ('пациент', 0.039696857), ('ген', 0.037584525), ('тело', 0.026724953), ('ткань', 0.02626402), ('учёный', 0.025934309), ('организм', 0.024928559), ('врач', 0.024352267)]


[('аккумулятор', 0.11015807), ('батарея', 0.07285596), ('зарядка', 0.056186624), ('ток', 0.0367697), ('напряжение', 0.032325253), ('батарейка', 0.029811667), ('заряд', 0.02496348), ('ячейка', 0.02082715), ('заряжать', 0.018348098), ('передатчик', 0.0142507795)]


[('спутник', 0.053409655), ('загадка', 0.04987625), ('орбита', 0.04790644), ('космический', 0.04469892), ('пуск', 0.042350158), ('ракета', 0.035754383), ('луна', 0.028792748), ('аппарат', 0.027439808), ('кольцо', 0.026169492), ('миссия', 0.02456002)]


Метрики модели:

In [82]:
lda3.log_perplexity(corpus[:10000])

-15.047433662573363

In [85]:
coherence_model_lda3 = gensim.models.CoherenceModel(model=lda3, 
                                                  texts=texts, 
                                                   dictionary=dct, coherence='c_v')

topics = []
for topic_id, topic in lda3.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda3 = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dct, coherence='c_v')

In [86]:
coherence_model_lda3.get_coherence()

0.396010889312874

# Сравнение моделей:

Хорошие темы в 1-ой модели: 70: 0.044*"вселенная" + 0.032*"звезда" + 0.024*"галактика" + 0.018*"солнце" + 0.017*"волна" + 0.016*"расстояние" + 0.016*"теория" + 0.015*"энергия" + 0.015*"масса" + 0.015*"излучение"

Похожая тема во 2-ой, но хуже: 1: 0.053*"uber" + 0.045*"вселенная" + 0.039*"звезда" + 0.037*"галактика" + 0.036*"поездка" + 0.030*"переводчик" + 0.025*"пассажир" + 0.019*"телескоп" + 0.018*"выручка" + 0.014*"небо"

Темы, появившиеся во 2-ой модели:

98: 0.099*"усилитель" + 0.087*"звук" + 0.049*"музыка" + 0.035*"коннектор" + 0.028*"наушник" + 0.024*"звучание" + 0.020*"музыкальный" + 0.016*"нарушать" + 0.015*"громкость" + 0.015*"создатель"


52: 0.110*"аккумулятор" + 0.073*"батарея" + 0.056*"зарядка" + 0.037*"ток" + 0.032*"напряжение" + 0.030*"батарейка" + 0.025*"заряд" + 0.021*"ячейка" + 0.018*"заряжать" + 0.014*"передатчик"

Метрики: 

Без tfidf: перплексия -7.78874924421697, когерентность 0.5474442339522236

С tfidf: перплексия -15.047433662573363, когерентность 0.396010889312874


# NMF

In [1]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [10]:
stexts = [' '.join(text) for text in texts]

In [56]:
vectorizer = TfidfVectorizer(max_features=2500, min_df=10, max_df=0.3, ngram_range=(1,3))
X = vectorizer.fit_transform(stexts)

In [57]:
model = NMF(n_components=100)

In [58]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=100, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [59]:
model.components_.shape

(100, 2500)

In [60]:
model.transform(X).shape

(4121, 100)

In [61]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [62]:
feat_names = vectorizer.get_feature_names()

In [63]:
top_words = model.components_.argsort()[:,:-7:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 какой то  что то  деньга  думать  вообще  жизнь
1 алгоритм  значение  точка  координата  переменный  матрица
2 игра  игровой  играть  движок  unity  персонаж
3 марс  аппарат  планета  земля  луна  космический
4 устройство  умный  мобильный устройство  девайс  гаджет  мобильный
5 public  void  public void  private  string  static
6 смартфон  экран  аккумулятор  apple  дисплей  ноутбук
7 the  to  of  and  in  is
8 модель  регистратор  моделирование  линейка  контекст  gps
9 уязвимость  обновление  пароль  патч  злоумышленник  безопасность
10 ракета  спутник  двигатель  космический  орбита  носитель
11 css  javascript  js  angular  html  es
12 доклад  конференция  выступление  тема  слайд  рассказать
13 браузер  страница  chrome  вкладка  расширение  веб
14 процессор  intel  память  ядро  гб  производительность
15 обучение  нейросеть  машинный  машинный обучение  нейронный  нейронный сеть
16 пакет  sudo  git  install  репозиторий  echo
17 книга  читать  чтение  автор  идея  прочитать
18

In [64]:
model.reconstruction_err_

52.515796643990484

# Сравнение с LDA:

В темах чаще появляются n-граммы (wi fi, резервный копия, программный обеспечение, виртуальный реальность и т.д.)

Темы, схожие с LDA:

3 марс  аппарат  планета  земля  луна  космический

93 квантовый  частица  излучение  пространство  теория  физика

70 диск  запись  ssd  жёсткий диск  жёсткий  накопитель

83 клетка  ген  организм  заболевание  тело  вещество

Новые темы:

17 книга  читать  чтение  автор  идея  прочитать

25 ия  интеллект  искусственный интеллект  искусственный  нейросеть  агент

52 автомобиль  машина  дорога  технология  безопасность  метр

97 учёный  исследование  научный  исследователь  университет  наука